In [ ]:
# Multi-tenant configuration: list each client's dataset and settings
clients = [
    {"id": "clientA", "train_key": "data/clientA-training.csv", "target": "target", "problem": "BinaryClassification"},
    {"id": "clientB", "train_key": "data/clientB-training.csv", "target": "target", "problem": "BinaryClassification"},
    # Add more clients as needed
]
instance_type = "ml.m5.large"
region = boto3.Session().region_name
role_arn = boto3.client("sts").get_caller_identity()["Arn"]

sm = boto3.client("sagemaker", region_name=region)

def train_and_deploy(client_cfg):
    cid = client_cfg["id"]
    job_name = f"autopilot-{cid}-{int(time.time())}"
    # Start Autopilot job for the client
    sm.create_auto_ml_job_v2(
        AutoMLJobName=job_name,
        AutoMLJobInputDataConfig=[{
            "ChannelType": "training",
            "ContentType": "text/csv;header=present",
            "DataSource": {"S3DataSource": {"S3DataType": "S3Prefix", "S3Uri": f"s3://{bucket}/{client_cfg['train_key']}"}}
        }],
        OutputDataConfig={"S3OutputPath": f"s3://{bucket}/autopilot-output/{cid}"},
        RoleArn=role_arn,
        AutoMLProblemTypeConfig={
            "TabularJobConfig": {
                "TargetAttributeName": client_cfg["target"],
                "ProblemType": client_cfg["problem"],
                "CompletionCriteria": {"MaxCandidates": 3}
            }
        }
    )
    print(f"Started AutoML job for {cid}: {job_name}")
    # Wait for completion (simplified polling)
    while True:
        desc = sm.describe_auto_ml_job_v2(AutoMLJobName=job_name)
        status = desc["AutoMLJobStatus"]
        if status in ["Completed", "Failed", "Stopped"]:
            break
        time.sleep(60)
    if status != "Completed":
        print(f"AutoML job for {cid} did not complete (status: {status})")
        return
    best = desc["BestCandidate"]
    model_name = f"{cid}-model-{int(time.time())}"
    sm.create_model(
        ModelName=model_name,
        Containers=best["InferenceContainers"],
        ExecutionRoleArn=role_arn,
        Tags=[{"Key": "ClientId", "Value": cid}]
    )
    endpoint_config_name = f"{cid}-config-{int(time.time())}"
    sm.create_endpoint_config(
        EndpointConfigName=endpoint_config_name,
        ProductionVariants=[{
            "VariantName": "AllTraffic", "ModelName": model_name,
            "InstanceType": instance_type, "InitialInstanceCount": 1
        }],
        Tags=[{"Key": "ClientId", "Value": cid}]
    )
    endpoint_name = f"autopilot-endpoint-{cid}"
    try:
        sm.describe_endpoint(EndpointName=endpoint_name)
        sm.update_endpoint(EndpointName=endpoint_name, EndpointConfigName=endpoint_config_name)
        print(f"Updated endpoint {endpoint_name} for client {cid}")
    except sm.exceptions.ClientError:
        sm.create_endpoint(EndpointName=endpoint_name, EndpointConfigName=endpoint_config_name, 
                           Tags=[{"Key": "ClientId", "Value": cid}])
        print(f"Created endpoint {endpoint_name} for client {cid}")
    waiter = sm.get_waiter("endpoint_in_service")
    waiter.wait(EndpointName=endpoint_name)
    print(f"Endpoint {endpoint_name} is InService for client {cid}")

# Loop over clients to train and deploy each one
for client in clients:
    train_and_deploy(client)
